In [1]:
import os

import pandas as pd

from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.docstore.document import Document

import gradio as gr


In [2]:
model= ChatGroq(model_name="llama3-70b-8192",  groq_api_key="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")

In [3]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en"
)

In [16]:
# Load the data from a CSV file
df = pd.read_csv('/home/vardh/RAG_Chatbot_Mistral/collegereview2022.csv')


# List to hold document instances
documents = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Format the page content string
    page_content = f"College: {row['college']} \nReview: {row['review']}"
    # Create a Document instance
    doc = Document(page_content=page_content)
    # Append the Document instance to the list
    documents.append(doc)

vectorstore = FAISS.from_documents(
    documents, embedding=embeddings
)
retriever = vectorstore.as_retriever(search_kwargs = {'k':10})

In [ ]:
def get_context(message):

    context = "\n".join(doc.page_content for doc in retriever.get_relevant_documents(message))
    return context


In [ ]:
# Define the ChatPromptTemplate for user interaction
template = """Answer the following question from the context
    context = {context}
    question = {question}
"""
prompt = PromptTemplate(input_variables=["context","question"], template= template)

In [ ]:
import gradio as gr

def slow_echo(message, history):
    context = get_context(message)
    for chunk in model.stream(prompt.format(question=message, context= context)):
        yield chunk

gr.ChatInterface(slow_echo).launch()